In [16]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from PIL import Image
import pandas as pd

In [17]:
picsdir = 'static'
df = pd.read_csv("markup.csv")
datalen = len(df)

In [21]:
validation_size = 0.25
autotune = tf.data.experimental.AUTOTUNE
img_size = 299
batch_size = 32
dropout = 0.1
lr = 1e-3
decay_steps = 100
decay_rate = 0.96
epochs = 35

In [22]:
def get_image(filename):
    image = tf.image.decode_jpeg(tf.io.read_file(f"{picsdir}/{filename}"), channels=3)
    image = tf.cast(tf.image.resize_with_pad(image, img_size, img_size), dtype = tf.float32)
    return tf.keras.applications.inception_v3.preprocess_input(image)

In [23]:
x = [get_image(filename) for filename in df["Filename"].values]

In [24]:
y = [int(mark > 3) for mark in df["Mark"].values]

In [25]:
dataset = tf.data.Dataset.from_tensor_slices((x, y)).batch(batch_size).prefetch(buffer_size=autotune)

In [26]:
train_part = 0.75
split_index = int(datalen * train_part // batch_size)
trainset = dataset.take(split_index)
validset = dataset.skip(split_index)

In [27]:
model = tf.keras.models.load_model('models/InceptionV3.h5')
for layer in model.layers[300:]:
    layer.trainable = True

In [28]:
model= tf.keras.models.Sequential(
    [
        tf.keras.layers.Input(shape=(img_size, img_size, 3)),
        tf.keras.layers.experimental.preprocessing.RandomFlip(mode="horizontal_and_vertical"),
        model,
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(dropout, name='top_dropout'),
        tf.keras.layers.Dense(32, activation='elu'),
        tf.keras.layers.Dense(1,'sigmoid')
    ]
)

In [29]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=lr,
    decay_steps=decay_steps, decay_rate=decay_rate,staircase=True)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='accuracy', patience=5, restore_best_weights=True)

class Callbackss(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs = {}):
        if logs.get('loss') < 0.063:
            self.model.stop_training = True
            
call = Callbackss()

In [30]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

In [31]:
history=model.fit(trainset,validation_data=validset, epochs=epochs, verbose=1, callbacks=[early_stop, call], use_multiprocessing=True, workers=-1)

Epoch 1/35
60/60 [==============================] - 464s 8s/step - loss: 1.0778 - accuracy: 0.6073 - val_loss: 0.7262 - val_accuracy: 0.4195
Epoch 2/35
60/60 [==============================] - 462s 8s/step - loss: 0.7709 - accuracy: 0.6068 - val_loss: 2.4515 - val_accuracy: 0.5258
Epoch 3/35
60/60 [==============================] - 481s 8s/step - loss: 0.7889 - accuracy: 0.5755 - val_loss: 79.3841 - val_accuracy: 0.4164
Epoch 4/35
60/60 [==============================] - 481s 8s/step - loss: 0.7530 - accuracy: 0.5964 - val_loss: 0.8165 - val_accuracy: 0.5836
Epoch 5/35
60/60 [==============================] - 468s 8s/step - loss: 0.7143 - accuracy: 0.6266 - val_loss: 0.7650 - val_accuracy: 0.5851
Epoch 6/35
60/60 [==============================] - 422s 7s/step - loss: 0.6579 - accuracy: 0.6432 - val_loss: 0.7794 - val_accuracy: 0.5836
Epoch 7/35
60/60 [==============================] - 443s 7s/step - loss: 0.6475 - accuracy: 0.6401 - val_loss: 33.2304 - val_accuracy: 0.5866
Epoch 8/35


KeyboardInterrupt: 